In [165]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from randomtimestamp import randomtimestamp, random_date, random_time
from datetime import datetime, time
from datetime import timedelta
# Set figure size to (28,14)
plt.rcParams['figure.figsize'] = (28, 14)


In [ ]:
#Display the number of customers at checkout (any state) over time

def question_03(df):
    df = pd.DataFrame(df.groupby(['location']).count()['customer_no'])
    return print(df)

In [2]:
#5 Calculate the total number of customers present in the supermarket over time.

def question_05(df):
    df = pd.DataFrame(df.groupby('timestamp').count()['customer_no'])
    sns.lineplot(data=df)

In [3]:
def question_04(df):
    df.reset_index(inplace=True)
    dict = {}
    # df_first = pd.DataFrame(columns=df.columns)
    # df_second = pd.DataFrame(columns=df.columns)
    for i in range(1,df['customer_no'].max(),1):
        #print(i)
        print(df[df['customer_no'] == i].iloc[0, 0])
        a = df[df['customer_no'] == i].iloc[-1, 0] - \
            df[df['customer_no'] == i].iloc[0, 0]
        dict[i] = a
        # df_first = df_first.append(a)
        # #print(df_first.head())
        # b = df[df['customer_no'] == i].iloc[1:]
        # df_second = df_second.append(b)
    # fig, axs = plt.subplots(2)
    # sns.histplot(df_first, x="location", ax=axs[0])
    # sns.histplot(df_second, x='location', ax=axs[1])
    return dict

In [4]:
# dict = question_06(df_0)

In [5]:
# Create customer_id
def create_id(df, letters):
    df['customer_id'] = letters + '_' + df['customer_no'].astype(str)

In [6]:
# Function for sorting
def sort_df(df):
    df.sort_index(inplace=True)
    df.sort_values('customer_no', kind='stable', inplace=True)
    return df


In [7]:
# Function for filling
def filling_steps(df):
    df_n = df.groupby('customer_no').resample('1T').ffill()
    df_l = df_n[['customer_no', 'location']].set_index('customer_no')
    df_l.reset_index(inplace=True)
    return df_l


In [8]:
# Function for adding checkouts to customers without
def checkout(df):
    for i in range(1, df['customer_no'].max()+1, 1):
        if df[df['customer_no'] == i].iloc[-1, 1] != 'checkout':
            a = df[df['customer_no'] == i].iloc[0,0]
            a_df = pd.DataFrame({"customer_no":[a],'location':'checkout'})
            df = df.append(a_df, ignore_index=True)
    return sort_df(df)

In [9]:
# create Markov columns
def markov_columns(df):
    df["before"] = df['location']
    df['after'] = df['location'].shift(-1)
    df.loc[df['before'] == 'checkout', 'after'] = 'checkout'
    df.set_index('customer_id', inplace=True)
    df.drop(columns=['customer_no','location'], inplace=True)
    return df


In [10]:
df_0 = pd.read_csv('data/monday.csv', parse_dates=True, index_col=0, sep=';')
df_1 = pd.read_csv('data/tuesday.csv', parse_dates=True, index_col=0, sep=';')
df_2 = pd.read_csv('data/wednesday.csv', parse_dates=True, index_col=0, sep=';')
df_3 = pd.read_csv('data/thursday.csv', parse_dates=True, index_col=0, sep=';')
df_4 = pd.read_csv('data/friday.csv', parse_dates=True, index_col=0, sep=';')


In [11]:
df_lst = [df_0, df_1, df_2, df_3, df_4]
days = ['MO','TU','WE','TH','FR']

In [12]:
for i in range(len(df_lst)):
    df_lst[i] = sort_df(df_lst[i])
    df_lst[i] = filling_steps(df_lst[i])
    df_lst[i] = checkout(df_lst[i])
    create_id(df_lst[i],days[i])


In [13]:
df_0.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4884 entries, 2019-09-02 07:03:00 to 2019-09-02 21:50:00
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   customer_no  4884 non-null   int64 
 1   location     4884 non-null   object
dtypes: int64(1), object(1)
memory usage: 114.5+ KB


In [14]:
df_0.head(20)

customer_no  location
timestamp                                 
2019-09-02 07:03:00            1     dairy
2019-09-02 07:05:00            1  checkout
2019-09-02 07:03:00            2     dairy
2019-09-02 07:06:00            2  checkout
2019-09-02 07:04:00            3     dairy
2019-09-02 07:06:00            3  checkout
2019-09-02 07:04:00            4     dairy
2019-09-02 07:08:00            4  checkout
2019-09-02 07:04:00            5    spices
2019-09-02 07:05:00            5  checkout
2019-09-02 07:04:00            6    spices
2019-09-02 07:05:00            6     dairy
2019-09-02 07:17:00            6    spices
2019-09-02 07:18:00            6    drinks
2019-09-02 07:25:00            6    spices
2019-09-02 07:26:00            6     dairy
2019-09-02 07:27:00            6    spices
2019-09-02 07:29:00            6  checkout
2019-09-02 07:04:00            7    spices
2019-09-02 07:05:00            7    drinks

In [15]:
df_lst[1].tail(10)

customer_no  location customer_id
10114         1420    drinks     TU_1420
10115         1420    drinks     TU_1420
10116         1420    drinks     TU_1420
10117         1420  checkout     TU_1420
10118         1421     fruit     TU_1421
10119         1421    drinks     TU_1421
10120         1421  checkout     TU_1421
10121         1422    spices     TU_1422
10122         1422     dairy     TU_1422
10124         1422  checkout     TU_1422

In [16]:
#we create a list with every single df in order to concatenate them together
df = pd.concat(df_lst)
df.head(50)


customer_no  location customer_id
0             1     dairy        MO_1
1             1     dairy        MO_1
2             1  checkout        MO_1
3             2     dairy        MO_2
4             2     dairy        MO_2
5             2     dairy        MO_2
6             2  checkout        MO_2
7             3     dairy        MO_3
8             3     dairy        MO_3
9             3  checkout        MO_3
10            4     dairy        MO_4
11            4     dairy        MO_4
12            4     dairy        MO_4
13            4     dairy        MO_4
14            4  checkout        MO_4
15            5    spices        MO_5
16            5  checkout        MO_5
17            6    spices        MO_6
18            6     dairy        MO_6
19            6     dairy        MO_6
20            6     dairy        MO_6
21            6     dairy        MO_6
22            6     dairy        MO_6
23            6     dairy        MO_6
24            6     dairy        MO_6
25            6     dairy        MO_6
26            6     dairy        MO_6
27            6     dairy        MO_6
28            6     dairy        MO_6
29            6     dairy        MO_6
30            6    spices        MO_6
31            6    drinks        MO_6
32            6    drinks        MO_6
33            6    drinks        MO_6
34            6    drinks        MO_6
35            6    drinks        MO_6
36            6    drinks        MO_6
37            6    drinks        MO_6
38            6    spices        MO_6
39            6     dairy        MO_6
40            6    spices        MO_6
41            6    spices        MO_6
42            6  checkout        MO_6
43            7    spices        MO_7
44            7    drinks        MO_7
45            7    drinks        MO_7
46            7    drinks        MO_7
47            7    drinks        MO_7
48            7    spices        MO_7
49            7    spices        MO_7

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53931 entries, 0 to 11075
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   customer_no  53931 non-null  int64 
 1   location     53931 non-null  object
 2   customer_id  53931 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.6+ MB


In [18]:
df.tail(30)

customer_no  location customer_id
11044         1500     fruit     FR_1500
11045         1500     fruit     FR_1500
11046         1500     dairy     FR_1500
11070         1500  checkout     FR_1500
11047         1501     fruit     FR_1501
11048         1501     fruit     FR_1501
11049         1501  checkout     FR_1501
11050         1502     fruit     FR_1502
11051         1502  checkout     FR_1502
11052         1503     dairy     FR_1503
11053         1503     dairy     FR_1503
11054         1503    drinks     FR_1503
11071         1503  checkout     FR_1503
11055         1504     fruit     FR_1504
11056         1504  checkout     FR_1504
11057         1505     fruit     FR_1505
11058         1505     dairy     FR_1505
11072         1505  checkout     FR_1505
11059         1506     dairy     FR_1506
11073         1506  checkout     FR_1506
11060         1507     dairy     FR_1507
11061         1507     dairy     FR_1507
11062         1507  checkout     FR_1507
11063         1508     dairy     FR_1508
11064         1508     dairy     FR_1508
11065         1508  checkout     FR_1508
11066         1509    drinks     FR_1509
11074         1509  checkout     FR_1509
11067         1510    spices     FR_1510
11075         1510  checkout     FR_1510

In [19]:
# General eda 
df['customer_id'].value_counts()

TH_1047    53
FR_102     53
WE_710     52
MO_666     52
FR_1238    50
           ..
TU_420      2
MO_617      2
WE_457      2
TU_1037     2
MO_1199     2
Name: customer_id, Length: 7445, dtype: int64

In [20]:
df_mar = markov_columns(df)
df_mar.head()


before     after
customer_id                    
MO_1            dairy     dairy
MO_1            dairy  checkout
MO_1         checkout  checkout
MO_2            dairy     dairy
MO_2            dairy     dairy

In [22]:
df_mar.tail(50)

before     after
customer_id                    
FR_1497         dairy     dairy
FR_1497         dairy     dairy
FR_1497         dairy  checkout
FR_1497      checkout  checkout
FR_1498         fruit     fruit
FR_1498         fruit  checkout
FR_1498      checkout  checkout
FR_1499         dairy     dairy
FR_1499         dairy     dairy
FR_1499         dairy     dairy
FR_1499         dairy     dairy
FR_1499         dairy  checkout
FR_1499      checkout  checkout
FR_1500         dairy    spices
FR_1500        spices    spices
FR_1500        spices    spices
FR_1500        spices    spices
FR_1500        spices     fruit
FR_1500         fruit     fruit
FR_1500         fruit     fruit
FR_1500         fruit     fruit
FR_1500         fruit     dairy
FR_1500         dairy  checkout
FR_1500      checkout  checkout
FR_1501         fruit     fruit
FR_1501         fruit  checkout
FR_1501      checkout  checkout
FR_1502         fruit  checkout
FR_1502      checkout  checkout
FR_1503         dairy     dairy
FR_1503         dairy    drinks
FR_1503        drinks  checkout
FR_1503      checkout  checkout
FR_1504         fruit  checkout
FR_1504      checkout  checkout
FR_1505         fruit     dairy
FR_1505         dairy  checkout
FR_1505      checkout  checkout
FR_1506         dairy  checkout
FR_1506      checkout  checkout
FR_1507         dairy     dairy
FR_1507         dairy  checkout
FR_1507      checkout  checkout
FR_1508         dairy     dairy
FR_1508         dairy  checkout
FR_1508      checkout  checkout
FR_1509        drinks  checkout
FR_1509      checkout  checkout
FR_1510        spices  checkout
FR_1510      checkout  checkout

In [23]:
df_mar.to_csv('df_mar.csv')

In [24]:
P = pd.crosstab(df_mar['before'], df_mar['after'], normalize=0)
P

after     checkout     dairy    drinks     fruit    spices
before                                                    
checkout  1.000000  0.000000  0.000000  0.000000  0.000000
dairy     0.103466  0.736750  0.058569  0.049848  0.051367
drinks    0.215710  0.010899  0.598499  0.087909  0.086983
fruit     0.201605  0.095924  0.054847  0.596947  0.050677
spices    0.150685  0.193214  0.163109  0.090953  0.402039

In [132]:
states = ['checkout','dairy','drinks',"fruit",'spices']

In [133]:
# convery the transition matrix to a dictionary that maps state to a list of probabilities
# order such that the first value is transition probability to cold and the second value
# is transition probability to hot

probs = P.to_dict(orient='index')
#probs

In [134]:
# convert internal values, which is a dict to a list
for key in probs.keys():
    probs[key] = list(probs[key].values())

#probs

In [135]:
class Customer:
    """
    a single customer that moves through the supermarket
    in a MCMC simulation
    """

    def __init__(self, name, state, transition_probs, budget=100):
        self.name = name
        self.state = state
        self.transition_probs = transition_probs
        self.budget = budget

    def __repr__(self):
        return f'<Customer {self.name} in {self.state}>'

    def next_state(self):
        '''
        Propagates the customer to the next state.
        Returns nothing.
        '''
        #self.state = random.choice(['spices', 'drinks', 'fruit'])
        self.state = random.choices(states, weights=self.transition_probs[self.state])[0]
        return self.state

    def is_active(self):
        """Returns True if the customer has not reached the checkout yet."""
        return self.state != 'checkout'



In [186]:
bagigio = Customer('bagigio','dairy',probs)

In [187]:
lst_customers = [bagigio]

In [188]:
for i in lst_customers:
    i = 0   
    min_time = random_time(text=True, pattern='%H:%M:%S')
    # create datetime object from timestamp string
    given_time = datetime.strptime(min_time, "%H:%M:%S")
    print(f"{given_time.time()} -- {bagigio.name} is in {bagigio.state} section")

    while bagigio.is_active():    
        i+=1
        print(f"{(given_time + timedelta(minutes = i)).time()} -- {bagigio.name} is in {bagigio.next_state()} section")


15:17:11 -- bagigio is in dairy section
15:18:11 -- bagigio is in dairy section
15:19:11 -- bagigio is in dairy section
15:20:11 -- bagigio is in dairy section
15:21:11 -- bagigio is in dairy section
15:22:11 -- bagigio is in dairy section
15:23:11 -- bagigio is in dairy section
15:24:11 -- bagigio is in dairy section
15:25:11 -- bagigio is in dairy section
15:26:11 -- bagigio is in dairy section
15:27:11 -- bagigio is in drinks section
15:28:11 -- bagigio is in drinks section
15:29:11 -- bagigio is in checkout section


In [168]:
a = random.choices(states, weights=probs['fruit'])
a

['checkout']